## Создаём предметно-ориентированного чат-бота

Для создания чат-бота, который может отвечать на вопросы по какой-то конкретной теме или предметной области, используется подход Retrieval-Augmented Generation (RAG).

Для начала установим необходимые библиотеки. Мы будем использовать библиотеку LangChain, поскольку она содержит в себе все необходимые компоненты для создания таких чат-ботов:
* общение с языковыми моделями
* векторная база данных
* эмбеддинги

In [1]:
%pip install langchain langchain_community yandexcloud gigachat chromadb langchain_chroma huggingface_hub sentence_transformers telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.5 MB/s 

Мы будем строить бота-консультанта для Школы дизайна ВШЭ. Для построения бота нужна текстовая база знаний, содержащая всю необходимую информацию. В идеале она должны быть разбита не небольшие кусочки, каждый из которых содержит законченный фрагмент информации.

Возьмём текст с сайта ШД, немного причёсанный и разбитый на фрагменты символами `//`:

In [ ]:
!wget https://github.com/shwars/ai-for-creatives/raw/refs/heads/main/data/kb.txt

--2026-01-19 10:09:36--  https://github.com/shwars/ai-for-creatives/raw/refs/heads/main/data/openbac.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shwars/ai-for-creatives/refs/heads/main/data/openbac.txt [following]
--2026-01-19 10:09:36--  https://raw.githubusercontent.com/shwars/ai-for-creatives/refs/heads/main/data/openbac.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36170 (35K) [text/plain]
Saving to: ‘openbac.txt’

openbac.txt         100%[===================>]  35.32K  --.-KB/s    in 0.01s   

2026-01-19 10:09:36 (2.87 MB/s) - ‘openbac.txt’ saved [36170/36170]



In [2]:
!head kb.txt

# Тема базы знаний: Запись в поликлинику в РФ — ОМС, прикрепление, Госуслуги/ЕМИАС, сроки, права пациента
# Версия: черновик для RAG-бота (сжатая справочная база)
# Источники (основные):
# - Госуслуги: запись к врачу / полис ОМС / меддокументы
# - ЕМИАС: FAQ по записи (Москва)
# - Mos.ru: запись к врачу (Москва)
# - 323-ФЗ: выбор врача и медорганизации
# - Справки по срокам ожидания (ОМС): региональные ТФОМС/страховые (как ориентиры) + программа госгарантий (общая логика)



В качестве языковой модели будем использовать Gigachat, поэтому нам потребуется ключ, который можно получить [по инструкции](https://developers.sber.ru/docs/ru/gigachat/quickstart/ind-using-api).

Импортируем необходимые библиотеки и создадим объекты:
* GPT для вызова Gigachat
* embeddings для вычисления семантических эмбеддингов текста. Для этого будем использовать мультиязычную модель [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large).

> Для вычисления эмбеддингов было бы идеально тоже использовать облачный сервис от Gigachat или Yandex, но Gigachat требует оплату за его использование.

> Модель эмбеддингов будет вычисляться внутри Colab, поэтому если не включить GPU - на индексирование всего текста может потребоваться 2-3 минуты.

In [3]:
from langchain_chroma import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models.gigachat import GigaChat
from google.colab import userdata
import warnings
warnings.filterwarnings('ignore')

gigachat_creds = userdata.get('gigachat_creds')

GPT = GigaChat(
    credentials=gigachat_creds,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    streaming=False,
    verify_ssl_certs=False,
)

embeddings = HuggingFaceEmbeddings(
    model_name = "intfloat/multilingual-e5-large"
)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Загрузим документ и разобьем его на фрагменты:

In [4]:
doc = open('kb.txt',encoding='utf-8').read()
docs = doc.split('\\')
print(f"Всего фрагментов: {len(docs)}")
print(f"Макс длина фрагмента: {max(map(len,docs))}")

Всего фрагментов: 1
Макс длина фрагмента: 9680


Поскольку длина макс фрагмента не превышает 1500 символов (500 токенов), то эти фрагменты можно дополнительно не разбивать на части. С учётом такой длины мы можем добавлять в запрос 3-5 найденных фрагментов текста.

Посмотрим, как работает вычисление эмбеддингов:

In [5]:
res = embeddings.embed_documents(docs[0])
print(f"Длина эмбеддингов: {len(res[0])}")

Длина эмбеддингов: 1024


Для хранения всех документов, проиндексированных по эмбеддингам, будем использовать вектоную базу данных, например, ChromaDB. При добавлении текстов в эту базу они будут автоматически проиндексированы:

In [6]:
db = Chroma('vecstore',embeddings,'./db')
db.add_texts(docs)

['914af932-8639-4962-8013-56e1fe3dd5c2']

Предположим, мы хотим найти среди документов ответ на вопрос **Сколько баллов ЕГЭ нужно для поступления в школу дизайна?**. Для этого определим объект `retriever`, и передадим ему параметр - количество документов, которые надо найти. После этого поиск сводится к простому вызову `retriever.invoke`

In [7]:
q = "Какие документы нужны для записи к врачу"

retriever = db.as_retriever(search_kwargs={"k": 3 })
retriever.invoke(q)


[Document(id='914af932-8639-4962-8013-56e1fe3dd5c2', metadata={}, page_content='# Тема базы знаний: Запись в поликлинику в РФ — ОМС, прикрепление, Госуслуги/ЕМИАС, сроки, права пациента\n# Версия: черновик для RAG-бота (сжатая справочная база)\n# Источники (основные):\n# - Госуслуги: запись к врачу / полис ОМС / меддокументы\n# - ЕМИАС: FAQ по записи (Москва)\n# - Mos.ru: запись к врачу (Москва)\n# - 323-ФЗ: выбор врача и медорганизации\n# - Справки по срокам ожидания (ОМС): региональные ТФОМС/страховые (как ориентиры) + программа госгарантий (общая логика)\n\n================================================================================\n1) БАЗОВЫЕ ПОНЯТИЯ: ОМС, полис, прикрепление\n================================================================================\n\nОМС (обязательное медицинское страхование) — система, которая даёт гражданам право на бесплатную медпомощь в рамках программы госгарантий. Полис ОМС — подтверждение, что человек застрахован и имеет право получать помощь п

Реализуем самую главную функцию ответа на вопрос! Она работает следующим образом:

1. Вызываем `retriever` и получаем 3 релевантных фрагмента текста
2. Объединяем их вместе в контекст `context`
3. Передаём GPT-модели промпт, включающий в себя исходный вопрос и контекст. Модель в этом случае сама смотрит на контекст и находит в нём нужную информацию.

In [8]:
def answer(q):
  res = retriever.invoke(q)
  context = '\n'.join(x.page_content for x in res)
  prompt = f"""
  Прочитай следующий текст и используй его при ответе на вопрос далее.
  Используй только информацию из текста. Если в тексте нет ответа на данный вопрос,
  напиши, что не знаешь. Вот текст:\n{context}\nВопрос: {q}"""
  return GPT.invoke(prompt).content

answer(q)

'Для записи к врачу обычно требуются следующие документы и сведения:\n\n1. **Персональные данные**:\n   - ФИО\n   - Дата рождения\n\n2. **Данные полиса ОМС**:\n   - Номер полиса ОМС (чаще всего нужен именно номер)\n\n3. **Дополнительно**:\n   - Иногда требуется СНИЛС (Страховой номер индивидуального лицевого счёта). \n   - Для детей младше 14 лет дополнительно указывается свидетельство о рождении.\n\nЕсли запись происходит через цифровые сервисы (Госуслуги, ЕМИАС, mos.ru), то потребуется также учетная запись и соответствие данных реестра прикрепленных пациентов.'

А теперь оформим это всё в виде телеграм-бота! Для этого используем библиотеку `telebot`. Чтобы создать бота нам нужно сначала получить специальный токен, пообщавшись в telegram со специальным ботом [@botfather](http://t.me/botfather). Полученный токен сохраните в секретах Google Colab.

Код ниже работает в режиме поллинга - он опрашивает сервера telegram на предмет наличия сообщений, и вызывает функцию `handle_message`, если сообщение пришло. Бот в телеграме будет работать только до тех пор, пока следующая ячейка выполняется.

In [1]:
import telebot

telegram_token = userdata.get('tg_token')

bot = telebot.TeleBot(telegram_token)

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    # Отправляем приветственное сообщение
    bot.send_message(message.chat.id,
                     'Привет, я бот, который знает всё запись в поликлинику. Спрашивай!')

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    ans = answer(message.text)
    bot.send_message(message.chat.id, ans)

# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

ModuleNotFoundError: No module named 'telebot'